Go to http://jalammar.github.io/illustrated-bert/ and read about bert pretraining, heads.


In [1]:
#for running in colab
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate datasets pandas

In [2]:
import pandas as pd
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [3]:
#let's work with datasets
from datasets import load_dataset
from datasets import Dataset

df=df.rename(columns={0: 'text', 1: 'label'})

C:\Users\nico_\anaconda3\envs\is\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df

,text,label
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1
...,...,...
6915,"painful , horrifying and oppressively tragic ,...",1
6916,take care is nicely performed by a quintet of ...,0
6917,"the script covers huge , heavy topics in a bla...",0
6918,a seriously bad film with seriously warped log...,0


In [5]:

dataset = Dataset.from_pandas(df)
splitted  = dataset.train_test_split(test_size=0.25)


In [6]:
splitted

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5190
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1730
    })
})

In [7]:
splitted['train'], splitted['test']

(Dataset({
     features: ['text', 'label'],
     num_rows: 5190
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 1730
 }))

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=100, truncation=True)

tokenized_splitted = splitted.map(tokenize_function, batched=True, batch_size=32)
#tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)

In [9]:
tokenized_splitted

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5190
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1730
    })
})

In [10]:
#examples from the dataset
tokenized_splitted['train'][1]['label'], tokenized_splitted['train'][1]['input_ids'], 

(0,
 [101,
  1996,
  2334,
  2063,
  3464,
  2521,
  2062,
  5875,
  2084,
  1996,
  2466,
  2012,
  2192,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0])

In [11]:
len(tokenized_splitted['train'][1]['input_ids']), len(tokenized_splitted['train'][1]['attention_mask'])
#why do we have here 100?

(100, 100)

In [12]:
from transformers import AutoModelForSequenceClassification
#create a model for sequence classification from distilbert-base pretrained model
#look at the model's head
model1 = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [13]:
model1, sum(p.numel() for p in model1.parameters() if p.requires_grad)

(DistilBertForSequenceClassification(
   (distilbert): DistilBertModel(
     (embeddings): Embeddings(
       (word_embeddings): Embedding(30522, 768, padding_idx=0)
       (position_embeddings): Embedding(512, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (transformer): Transformer(
       (layer): ModuleList(
         (0-5): 6 x TransformerBlock(
           (attention): MultiHeadSelfAttention(
             (dropout): Dropout(p=0.1, inplace=False)
             (q_lin): Linear(in_features=768, out_features=768, bias=True)
             (k_lin): Linear(in_features=768, out_features=768, bias=True)
             (v_lin): Linear(in_features=768, out_features=768, bias=True)
             (out_lin): Linear(in_features=768, out_features=768, bias=True)
           )
           (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
           (ffn): FFN(
             (dropout): Dropout(p=

In [14]:
import torch
#device = torch.device("cpu") if torch.cuda.is_available() else torch.device("cuda")
#model1.to(device)
#device

In [15]:
from transformers import TrainingArguments, Trainer

In [16]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import numpy as np
torch.device("cpu")
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=128, # for gpu try with 32
    per_device_eval_batch_size=32,
    warmup_steps=200,
    weight_decay=0.01,
    evaluation_strategy="steps",
    logging_strategy="steps",
    eval_steps=50,
    logging_dir='./logs',
    logging_steps = 50
    
)

trainer = Trainer(
    model=model1,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_splitted['train'], 
    eval_dataset=tokenized_splitted['test'],
    #place_model_on_device=True
)

In [17]:
#from transformers import Trainer

#trainer = Trainer(
#    model=model, args=training_args, train_dataset=tokenized_splitted['train'], 
#    eval_dataset=tokenized_splitted['test']
#)

In [18]:
trainer.train()

C:\Users\nico_\anaconda3\envs\is\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.654800,0.485670,0.824855,0.845329,0.803104,0.892241
100,0.345400,0.301987,0.887283,0.891486,0.921749,0.863147


TrainOutput(global_step=123, training_loss=0.4533060934485459, metrics={'train_runtime': 5559.3708, 'train_samples_per_second': 2.801, 'train_steps_per_second': 0.022, 'total_flos': 402835429116000.0, 'train_loss': 0.4533060934485459, 'epoch': 3.0})

In [19]:
trainer.evaluate()

{'eval_loss': 0.29716822504997253,
 'eval_accuracy': 0.8872832369942196,
 'eval_f1': 0.8990160538581047,
 'eval_precision': 0.8654037886340977,
 'eval_recall': 0.9353448275862069,
 'eval_runtime': 176.7326,
 'eval_samples_per_second': 9.789,
 'eval_steps_per_second': 0.311,
 'epoch': 3.0}

In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir logs

In [23]:
#sentence="I am so tired and I still do not know if it works."
#sentence="I am not sad."
sentence="The movie was great."
sentence2 = "The director failed this time, no words can describe the feeling after the end of the movie" 
#sentence = "It's winter and it  is  snowing outside"
#sentence = "It's winter and it is snowing outside"
tokenized = tokenizer([sentence, sentence2], padding="max_length", max_length=100, truncation=True, return_tensors='pt')

In [42]:
tokenized.keys()

dict_keys(['input_ids', 'attention_mask'])

In [31]:
preds=trainer.model(**tokenized)
preds

SequenceClassifierOutput(loss=None, logits=tensor([[-1.7715,  2.0054],
        [ 1.3742, -1.2954]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [41]:
predictions = torch.nn.functional.softmax(preds.logits, dim=-1)
predictions

tensor([[0.0224, 0.9776],
        [0.9352, 0.0648]], grad_fn=<SoftmaxBackward0>)

In [ ]:
#torch.cuda.empty_cache()

In [ ]:
from transformers import BartTokenizer, BartForCausalLM

import torch
#sentence = "It's winter and it is snowing outside"
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#model = GPT2LMHeadModel.from_pretrained('gpt2')


tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-1.3B')

inputs = tokenizer.encode("Hello, my dog is cute", return_tensors="pt")
outputs = model.generate(inputs, labels=inputs, do_sample=True, temperature=0.9, max_length=100)
tokenizer.decode(outputs[0])

#last_hidden_states = outputs.last_hidden_state


In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')
generator("EleutherAI has", do_sample=True, min_length=50)

In [ ]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids, 
    max_length = 20, 
    num_beams = 5, 
    no_repeat_ngram_size = 2, 
    num_return_sequences = 5, 
    early_stopping = True
)

In [ ]:
for i, beam_output in enumerate(beam_outputs):
      print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))


In [ ]:
from transformers import DistilBertModel, DistilBertForSequenceClassification,  DistilBertForTokenClassification, DistilBertForMaskedLM
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
model1 = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model2 = DistilBertForTokenClassification.from_pretrained('distilbert-base-uncased')
model3 = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')

In [ ]:
model

In [ ]:
model1

In [ ]:
model2

In [ ]:
model3

In [ ]:
from transformers import BartModel, BartForSequenceClassification,  BartForConditionalGeneration
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model10 = BartModel.from_pretrained('facebook/bart-large')
model11= BartForSequenceClassification.from_pretrained('facebook/bart-large')
model12 = BartForConditionalGeneration.from_pretrained('facebook/bart-large')


In [ ]:
model10

In [ ]:
model11

In [ ]:
model12